In [38]:
from qdrant_client import QdrantClient, models

In [39]:
client = QdrantClient(url="http://localhost:6333")

In [40]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
        payloads=[
            {"color": "red", "city": "London",},
            {"color": "green", "city": "Moscow"},
            {"color": "blue", "city": "Moscow"},
            {"color": "green", "city": "London"},
            {"color": "red", "city": "Berlin"},
            {"color": "blue", "city": "London"},
            {"color": "red", "city": "Shanghai"},
            {"color": "black", "city": "Guangzhou"},
            {"color": "red", "city": "Beijing"},
            {"color": "yellow", "city": "Changsha"},
            {"color": "red", "city": "London"},
            {"color": "green", "city": "Changsha"},
            {"color": "red", "city": "Hongkang"},
            {"color": "red", "city": "Shanghai"},
        ],
        vectors=[
            [0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2],
            [0.3, 0.3, 0.3],
            [0.4, 0.4, 0.4],
            [0.5, 0.5, 0.5],
            [0.6, 0.6, 0.6],
            [0.7, 0.7, 0.7],
            [0.8, 0.8, 0.8],
            [0.9, 0.9, 0.9],
            [1.0, 1.0, 1.0],
            [1.1, 1.1, 1.1],
            [1.2, 1.2, 1.2],
            [1.3, 1.3, 1.3],
            [1.4, 1.4, 1.4],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ]
    ),
    # limit: How many points to return
    limit=34,
    # If `True` - attach all payload
    # If `False` - do not attach any payload
    # If List of string - include only specified fields
    # If `PayloadSelector` - use explicit rules
    with_payload=True,  # 默认with_payload=True
    # If `True` - Attach stored vector to the search result.
    # If `False` - Do not attach vector.
    # If List of string - Attach only specified vectors.
    with_vectors=True,  # 默认with_vectors=True
)

([Record(id=1, payload={'city': 'London', 'color': 'red'}, vector=[0.1, 0.1, 0.1], shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'Berlin', 'color': 'red'}, vector=[0.5, 0.5, 0.5], shard_key=None, order_value=None),
  Record(id=7, payload={'city': 'Shanghai', 'color': 'red'}, vector=[0.7, 0.7, 0.7], shard_key=None, order_value=None),
  Record(id=9, payload={'city': 'Beijing', 'color': 'red'}, vector=[0.9, 0.9, 0.9], shard_key=None, order_value=None),
  Record(id=11, payload={'city': 'London', 'color': 'red'}, vector=[1.1, 1.1, 1.1], shard_key=None, order_value=None),
  Record(id=13, payload={'city': 'Hongkang', 'color': 'red'}, vector=[1.3, 1.3, 1.3], shard_key=None, order_value=None),
  Record(id=14, payload={'city': 'Shanghai', 'color': 'red'}, vector=[1.4, 1.4, 1.4], shard_key=None, order_value=None)],
 None)

In [43]:
# 分页查询
scroll0 = client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ]
    ),
    limit=2,
    # offset: If provided - skip points with ids less than given `offset`
    offset=7
)
print(scroll0, end='\n\n')  # scroll0[1]=11

scroll1 = client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ]
    ),
    limit=2,
    offset=scroll0[1]
)
print(scroll1, end='\n\n')  # scroll1[1]=14

scroll2 = client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ]
    ),
    limit=2,
    offset=scroll1[1]
)
print(scroll2)
print(scroll2[1])  # scroll2[1]=None,表示查询完毕

([Record(id=7, payload={'city': 'Shanghai', 'color': 'red'}, vector=None, shard_key=None, order_value=None), Record(id=9, payload={'city': 'Beijing', 'color': 'red'}, vector=None, shard_key=None, order_value=None)], 11)

([Record(id=11, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None), Record(id=13, payload={'city': 'Hongkang', 'color': 'red'}, vector=None, shard_key=None, order_value=None)], 14)

([Record(id=14, payload={'city': 'Shanghai', 'color': 'red'}, vector=None, shard_key=None, order_value=None)], None)
None
